<a href="https://colab.research.google.com/github/PrasanthV454/Genetic-Algoritms/blob/master/EfficientNet_B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
import os
import glob
import shutil
import sys
import numpy as np
from skimage.io import imread
import matplotlib.pyplot as plt
from IPython.display import Image

batch_size = 48
width=150
height=150
epochs=20
NUM_TRAIN=2000
NUM_TEST=1000
dropout_rate=0.2
input_shape=(height,width,3)

!git clone https://github.com/Tony607/efficientnet_keras_transfer_learning


TensorFlow 1.x selected.
fatal: destination path 'efficientnet_keras_transfer_learning' already exists and is not an empty directory.


In [2]:
%cd efficientnet_keras_transfer_learning/

/content/efficientnet_keras_transfer_learning


In [3]:
from efficientnet import EfficientNetB0 as Net
from efficientnet import center_crop_and_resize, preprocess_input
conv_base=Net(weights='imagenet',include_top=False,input_shape=input_shape)

Using TensorFlow backend.



Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
16719872/16717576 [==============================] - 2s 0us/step


In [4]:
!wget https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip

ERROR! Session/line number was not unique in database. History logging moved to new session 60
--2020-03-31 17:17:00--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.193.24.126, 2600:1417:76:586::e59, 2600:1417:76:58e::e59
Connecting to download.microsoft.com (download.microsoft.com)|23.193.24.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824894548 (787M) [application/octet-stream]
Saving to: ‘kagglecatsanddogs_3367a.zip’

kagglecatsanddogs_3 100%[===================>] 786.68M   203MB/s    in 4.0s    

2020-03-31 17:17:05 (196 MB/s) - ‘kagglecatsanddogs_3367a.zip’ saved [824894548/824894548]



In [6]:
!unzip -qq kagglecatsanddogs_3367a.zip -d dog_vs_cat

replace dog_vs_cat/PetImages/Cat/0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace dog_vs_cat/PetImages/Cat/1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace dog_vs_cat/PetImages/Cat/10.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [7]:
!ls dog_vs_cat/PetImages/

Cat  Dog


In [8]:
original_dataset_dir='./dog_vs_cat/PetImages'
cat_images = glob.glob(os.path.join(original_dataset_dir, "Cat", '*.jpg'))
dog_images = glob.glob(os.path.join(original_dataset_dir, "Dog", '*.jpg'))
print(len(cat_images))
print(len(dog_images))

12500
12500


In [0]:
base_dir = './data/dog_vs_cat_small'
os.makedirs(base_dir, exist_ok=True)
train_dir = os.path.join(base_dir, 'train')
os.makedirs(train_dir, exist_ok=True)
validation_dir = os.path.join(base_dir, 'validation')
os.makedirs(validation_dir, exist_ok=True)
test_dir = os.path.join(base_dir, 'test')
os.makedirs(test_dir, exist_ok=True)
train_cats_dir = os.path.join(train_dir, 'cats')
os.makedirs(train_cats_dir, exist_ok=True)
train_dogs_dir = os.path.join(train_dir, 'dogs')
os.makedirs(train_dogs_dir, exist_ok=True)
validation_cats_dir = os.path.join(validation_dir, 'cats')
os.makedirs(validation_cats_dir, exist_ok=True)
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.makedirs(validation_dogs_dir, exist_ok=True)
test_cats_dir = os.path.join(test_dir, 'cats')
os.makedirs(test_cats_dir, exist_ok=True)
test_dogs_dir = os.path.join(test_dir, 'dogs')
os.makedirs(test_dogs_dir, exist_ok=True)

fnames = cat_images[:NUM_TRAIN//2]
for fname in fnames:
    dst = os.path.join(train_cats_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)

offset = NUM_TRAIN//2

fnames = cat_images[offset:offset + NUM_TEST // 2]
for fname in fnames:
    dst = os.path.join(validation_cats_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)

offset = offset + NUM_TEST // 2

fnames = cat_images[offset:offset + NUM_TEST // 2]
for fname in fnames:
    dst = os.path.join(test_cats_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)

fnames = dog_images[:NUM_TRAIN//2]
for fname in fnames:
    dst = os.path.join(train_dogs_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)


offset = NUM_TRAIN//2

fnames = dog_images[offset:offset + NUM_TEST // 2]
for fname in fnames:
    dst = os.path.join(validation_dogs_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)

offset = offset + NUM_TEST // 2

fnames = dog_images[offset:offset + NUM_TEST // 2]
for fname in fnames:
    dst = os.path.join(test_dogs_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)

In [10]:
print('total training cat images:', len(os.listdir(train_cats_dir)))
print('total training dog images:', len(os.listdir(train_dogs_dir)))
print('total validation cat images:', len(os.listdir(validation_cats_dir)))
print('total validation dog images:', len(os.listdir(validation_dogs_dir)))
print('total test cat images:', len(os.listdir(test_cats_dir)))
print('total test dog images:', len(os.listdir(test_dogs_dir)))

total training cat images: 1000
total training dog images: 1000
total validation cat images: 500
total validation dog images: 500
total test cat images: 500
total test dog images: 500


In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [14]:
model =  models.Sequential()
model.add(conv_base)
model.add(layers.GlobalMaxPooling2D(name="gap"))
model.add(layers.Dense(2, activation='softmax', name="fc_out"))

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b0 (Model)      (None, 5, 5, 1280)        4049564   
_________________________________________________________________
gap (GlobalMaxPooling2D)     (None, 1280)              0         
_________________________________________________________________
fc_out (Dense)               (None, 2)                 2562      
Total params: 4,052,126
Trainable params: 4,010,110
Non-trainable params: 42,016
_________________________________________________________________


In [16]:
conv_base.trainable = False
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b0 (Model)      (None, 5, 5, 1280)        4049564   
_________________________________________________________________
gap (GlobalMaxPooling2D)     (None, 1280)              0         
_________________________________________________________________
fc_out (Dense)               (None, 2)                 2562      
Total params: 4,052,126
Trainable params: 2,562
Non-trainable params: 4,049,564
_________________________________________________________________


In [17]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

history= model.fit_generator(
    train_generator,
    steps_per_epoch=NUM_TRAIN//batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    verbose=1,
    use_multiprocessing=True,
    workers=4)


Epoch 1/20
41/41 [==============================] - 152s 4s/step - loss: 1.5810 - acc: 0.5323 - val_loss: 1.2855 - val_acc: 0.5790
Epoch 2/20
41/41 [==============================] - 151s 4s/step - loss: 1.3692 - acc: 0.5645 - val_loss: 1.1773 - val_acc: 0.6030
Epoch 3/20
41/41 [==============================] - 151s 4s/step - loss: 1.3524 - acc: 0.5712 - val_loss: 1.0903 - val_acc: 0.6120
Epoch 4/20
41/41 [==============================] - 151s 4s/step - loss: 1.3538 - acc: 0.5686 - val_loss: 1.0200 - val_acc: 0.6320
Epoch 5/20
41/41 [==============================] - 154s 4s/step - loss: 1.2659 - acc: 0.5738 - val_loss: 0.9605 - val_acc: 0.6450
Epoch 6/20
41/41 [==============================] - 152s 4s/step - loss: 1.2423 - acc: 0.5876 - val_loss: 0.9043 - val_acc: 0.6480
Epoch 7/20
41/41 [==============================] - 152s 4s/step - loss: 1.1208 - acc: 0.6148 - val_loss: 0.8576 - val_acc: 0.6620
Epoch 8/20
41/41 [==============================] - 153s 4s/step - loss: 1.1726 - a

In [18]:
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical')

result_loss,result_acc=model.evaluate(test_generator)
print(result_acc)

Found 1000 images belonging to 2 classes.
21/21 [==============================] - 40s 2s/step - loss: 0.6713 - acc: 0.7650
0.765
